In [1]:
!nvidia-smi

Fri Jan 21 10:38:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# %%writefile main.cu

In [ ]:
%%writefile main.cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <chrono>
#include <fstream>
#include <functional>
#include <iomanip>
#include <iostream>
#include <random>
#include <vector>

using namespace std;

const int N = 40000;
const int BS = 256;
__device__ const double G = 6.67e-11;


template<typename T>
ostream& operator<<(ostream& stream, const vector<T>& vec) {
    for (auto it = vec.begin(); it != vec.end(); ++it) {
        stream << *it;
    }
    return stream;
}

__global__ void plus_product(double* result, double* lhs, double tau, double* rhs, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        result[idx] = lhs[idx] + tau * rhs[idx];
    }
}

__global__ void plus_equal_product(double* result, double tau, double* rhs, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        result[idx] += tau * rhs[idx];
    }
}

__global__ void calc_k(double* result, double* pos_vel, double* masses, int n) {
    double eps = 1e-15;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        result[3 * idx + 0] = pos_vel[3 * n + 3 * idx + 0];
        result[3 * idx + 1] = pos_vel[3 * n + 3 * idx + 1];
        result[3 * idx + 2] = pos_vel[3 * n + 3 * idx + 2];
        double a_x = 0.0, a_y = 0.0, a_z = 0.0;
        double dist3, coef;
        for (int j = 0; j < n; ++j) {
            dist3 = pow(pow(pos_vel[3 * idx + 0] - pos_vel[3 * j + 0], 2) +
                                pow(pos_vel[3 * idx + 1] - pos_vel[3 * j + 1], 2) +
                                pow(pos_vel[3 * idx + 2] - pos_vel[3 * j + 2], 2),
                        3.0 / 2);
            coef = -G * masses[j] / max(dist3, eps);
            a_x += coef * (pos_vel[3 * idx + 0] - pos_vel[3 * j + 0]);
            a_y += coef * (pos_vel[3 * idx + 1] - pos_vel[3 * j + 1]);
            a_z += coef * (pos_vel[3 * idx + 2] - pos_vel[3 * j + 2]);
        }
        result[3 * n + 3 * idx + 0] = a_x;
        result[3 * n + 3 * idx + 1] = a_y;
        result[3 * n + 3 * idx + 2] = a_z;
    }
}

void runge_kutta_2(const vector<double>& init, const vector<double>& masses, double t_max, double tau,
                   int print_every_n, const string& file_name) {
    ofstream stream(file_name);
    int size = init.size();
    int n = masses.size();
    int step_num = 0;
    double t_cur = 0.0;
    vector<double> p_cur(init), p_mid(init), k(init);
    stream << setprecision(10) << t_cur << " " << p_cur << '\n';

    double *d_p_cur, *d_p_mid, *d_masses, *d_k;
    cudaMalloc(&d_p_cur, size * sizeof(double));
    cudaMalloc(&d_p_mid, size * sizeof(double));
    cudaMalloc(&d_masses, n * sizeof(double));
    cudaMalloc(&d_k, size * sizeof(double));

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    float elapsed_time;
    float exec_time = 0.0, copy_time = 0.0;

    cudaEventRecord(start);
    /* copy masses and p(t) to device */
    cudaMemcpy(d_p_cur, p_cur.data(), size * sizeof(double), cudaMemcpyHostToDevice);
    cudaMemcpy(d_masses, masses.data(), n * sizeof(double), cudaMemcpyHostToDevice);
    /* end of record */
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&elapsed_time, start, stop);
    copy_time += elapsed_time;

    while (t_cur < t_max) {
        t_cur += tau;

        cudaEventRecord(start);
        /* k(t) = { v(t), a(t) } */
        calc_k<<<(n + BS) / BS, BS>>>(d_k, d_p_cur, d_masses, n);
        /* p(t + tau/2) = p(t) + tau / 2 * k(t) */
        plus_product<<<(size + BS) / BS, BS>>>(d_p_mid, d_p_cur, tau / 2, d_k, size);
        /* k(t + tau/2) = { v(t + tau/2), a(t + tau/2) } */
        calc_k<<<(n + BS) / BS, BS>>>(d_k, d_p_mid, d_masses, n);
        /* p(t + tau) += tau * k(t + tau/2) */
        plus_equal_product<<<(size + BS) / BS, BS>>>(d_p_cur, tau, d_k, size);
        /* end of record */
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&elapsed_time, start, stop);
        exec_time += elapsed_time;

        cudaEventRecord(start);
        /* copy p(t + tau) to host */
        cudaMemcpy(p_cur.data(), d_p_cur, size * sizeof(double), cudaMemcpyDeviceToHost);
        /* end of record */
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&elapsed_time, start, stop);
        copy_time += elapsed_time;

        if (++step_num % print_every_n == 0) {
            stream << setprecision(10) << t_cur << " " << p_cur << '\n';
        }
    }

    cudaDeviceSynchronize();

    cudaFree(d_p_cur);
    cudaFree(d_p_mid);
    cudaFree(d_masses);
    cudaFree(d_k);
    cout << "Time exec: " << exec_time / 1000.0 << endl;
    cout << "Time copy: " << copy_time / 1000.0 << endl;
}

int main(int argc, char** argv) {
    /* part 1 data init */
    vector<double> input_data = {1.0, 0.0, 0.0, 0.0, 0.9, 0.0,
                                 0.0, 1.0, 0.0, -0.9, 0.0, 0.0,
                                 -1.0, 0.0, 0.0, 0.0, -0.9, 0.0,
                                 0.0, -1.0, 0.0, 0.9, 0.0, 0.0};
    vector<double> masses = {8810324116.227, 8810324116.227, 8810324116.227, 8810324116.227};
    vector<double> init(input_data.size());
    for (int i = 0; i < init.size() / 6; ++i) {
        init[3 * i + 0] = input_data[6 * i + 0];
        init[3 * i + 1] = input_data[6 * i + 1];
        init[3 * i + 2] = input_data[6 * i + 2];
        init[init.size() / 2 + 3 * i + 0] = input_data[6 * i + 3];
        init[init.size() / 2 + 3 * i + 1] = input_data[6 * i + 4];
        init[init.size() / 2 + 3 * i + 2] = input_data[6 * i + 5];
    }

    /* part 1 calc */
    // runge_kutta_2(init, masses, 20, 0.01, 10, "./cuda.txt");

    /* distributions init */
    std::random_device seeder;
    const auto seed = seeder.entropy() ? seeder() : time(nullptr);
    std::mt19937 rnd_gen(static_cast<std::mt19937::result_type>(seed));
    std::uniform_real_distribution<double> distr_pos(-10.0, 10.0);
    std::uniform_real_distribution<double> distr_vel(-1.0, 1.0);
    std::uniform_real_distribution<double> distr_mass(0.1, 10.0);
    auto rnd_pos = std::bind(distr_pos, rnd_gen);
    auto rnd_vel = std::bind(distr_vel, rnd_gen);
    auto rnd_mas = std::bind(distr_mass, rnd_gen);

    /* part 2 data init */
    init.resize(6 * N);
    for (int i = 0; i < 3 * N; ++i) {
        init[i] = rnd_pos();
    }
    for (int i = 3 * N; i < 6 * N; ++i) {
        init[i] = rnd_vel();
    }
    masses.resize(N);
    for (int i = 0; i < N; ++i) {
        masses[i] = rnd_mas();
    }

    cout << "N: " << N << "\n\n";

    /* part 2 calc (cuda) */
    auto t1 = chrono::high_resolution_clock::now();
    runge_kutta_2(init, masses, 0.1, 0.05, 1, "./cuda_10000.txt");
    auto t2 = chrono::high_resolution_clock::now();
    cout << "Time cuda: " << chrono::duration_cast<chrono::milliseconds>(t2 - t1).count() / 1000.0 << endl;

    return 0;
}

Overwriting main.cu


In [ ]:
!nvcc -arch=sm_75 -std=c++11 -O2 -o run -Wno-deprecated-gpu-targets main.cu
# sm37 (K80)
# sm75 (T4)
# -gencode arch=compute_52,code=sm_52

In [ ]:
!./run

N: 40000

Time exec: 21.2751
Time copy: 0.001752
Time cuda: 21.661


* N: 10000

Time exec: 5.41399<br>
Time copy: 0.000518016<br>

Time cuda: 5.724 (Tesla T4)<br>
Time seq: ?

* N: 20000

Time exec: 10.7356<br>
Time copy: 0.00101994<br>

Time cuda: 11.123 (Tesla T4)<br>
Time seq: ?

* N: 30000

Time exec: 15.9645<br>
Time copy: 0.00133862<br>
Time cuda: 16.421 (Tesla T4)<br>
Time seq: ?

In [2]:
%%writefile main1.cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <chrono>
#include <fstream>
#include <functional>
#include <iomanip>
#include <iostream>
#include <random>
#include <vector>

using namespace std;

int N = 10000;
int BS = 1024;
__device__ const double G = 6.67e-11;
__device__ const double EPS = 1e-15;


template<typename T>
ostream& operator<<(ostream& stream, const vector<T>& vec) {
    for (auto it = vec.begin(); it != vec.end(); ++it) {
        stream << " " << *it;
    }
    return stream;
}

__global__ void plus_product(double* result, double* lhs, double tau, double* rhs, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        result[idx] = lhs[idx] + tau * rhs[idx];
    }
}

__global__ void plus_equal_product(double* result, double tau, double* rhs, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        result[idx] += tau * rhs[idx];
    }
}

//__global__ void calc_k(double* result, double* pos_vel, double* masses, int n) {
//    int idx = blockIdx.x * blockDim.x + threadIdx.x;
//    if (idx < n) {
//        result[3 * idx + 0] = pos_vel[3 * n + 3 * idx + 0];
//        result[3 * idx + 1] = pos_vel[3 * n + 3 * idx + 1];
//        result[3 * idx + 2] = pos_vel[3 * n + 3 * idx + 2];
//        double a_x = 0.0, a_y = 0.0, a_z = 0.0;
//        double dist3, coef;
//        for (int j = 0; j < n; ++j) {
//            dist3 = pow(pow(pos_vel[3 * idx + 0] - pos_vel[3 * j + 0], 2) +
//                                pow(pos_vel[3 * idx + 1] - pos_vel[3 * j + 1], 2) +
//                                pow(pos_vel[3 * idx + 2] - pos_vel[3 * j + 2], 2),
//                        3.0 / 2);
//            coef = masses[j] / max(dist3, EPS);
//            a_x += coef * (pos_vel[3 * idx + 0] - pos_vel[3 * j + 0]);
//            a_y += coef * (pos_vel[3 * idx + 1] - pos_vel[3 * j + 1]);
//            a_z += coef * (pos_vel[3 * idx + 2] - pos_vel[3 * j + 2]);
//        }
//        result[3 * n + 3 * idx + 0] = -G * a_x;
//        result[3 * n + 3 * idx + 1] = -G * a_y;
//        result[3 * n + 3 * idx + 2] = -G * a_z;
//    }
//}

__device__ double sqr(double x) {
    return x * x;
}

__global__ void calc_k(double* result, double* pos_vel, double* masses, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        result[3 * idx] = pos_vel[3 * n + 3 * idx];
        result[3 * idx + 1] = pos_vel[3 * n + 3 * idx + 1];
        result[3 * idx + 2] = pos_vel[3 * n + 3 * idx + 2];

        /* copy all positions to shared memory */
        extern __shared__ double pos[];
        pos[3 * idx] = pos_vel[3 * idx];
        pos[3 * idx + 1] = pos_vel[3 * idx + 1];
        pos[3 * idx + 2] = pos_vel[3 * idx + 2];
        /* copy all masses to shared memory */
        extern __shared__ double mas[];
        mas[idx] = masses[idx];
        __syncthreads();
        /* copy [idx] position to register memory */
        double pos_idx[3] = {pos[3 * idx], pos[3 * idx + 1], pos[3 * idx + 2]};
        /* initialize [idx] acceleration on register memory */
        double acc[3] = {0.0, 0.0, 0.0};
        /* initialize coefs on register memory */
        double dist3, coef, diff[3];

        for (int j = 0; j < n; ++j) {
            diff[0] = pos[3 * j] - pos_idx[0];
            diff[1] = pos[3 * j + 1] - pos_idx[1];
            diff[2] = pos[3 * j + 2] - pos_idx[2];
            dist3 = pow(sqr(diff[0]) + sqr(diff[1]) + sqr(diff[2]), 1.5);
            coef = mas[j] / max(dist3, EPS);
            acc[0] += coef * diff[0];
            acc[1] += coef * diff[1];
            acc[2] += coef * diff[2];
        }
        result[3 * n + 3 * idx + 0] = G * acc[0];
        result[3 * n + 3 * idx + 1] = G * acc[1];
        result[3 * n + 3 * idx + 2] = G * acc[2];
    }
}

void runge_kutta_2(const vector<double>& init, const vector<double>& masses, double t_max, double tau,
                   int print_every_n, const string& file_name) {
    ofstream stream(file_name);
    int n = masses.size();
    int size = init.size();  // 6 * n
    int step_num = 0;
    double t_cur = 0.0;
    vector<double> p_cur(init);
    stream << setprecision(10) << t_cur << " " << p_cur << '\n';

    double *d_p_cur, *d_p_mid, *d_masses, *d_k;
    cudaMalloc(&d_p_cur, size * sizeof(double));
    cudaMalloc(&d_p_mid, size * sizeof(double));
    cudaMalloc(&d_masses, n * sizeof(double));
    cudaMalloc(&d_k, size * sizeof(double));

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    float elapsed_time;
    float exec_time = 0.0, copy_time = 0.0;

    cudaEventRecord(start);
    /* copy masses and p(t) to device */
    cudaMemcpy(d_p_cur, p_cur.data(), size * sizeof(double), cudaMemcpyHostToDevice);
    cudaMemcpy(d_masses, masses.data(), n * sizeof(double), cudaMemcpyHostToDevice);
    /* end of record */
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&elapsed_time, start, stop);
    copy_time += elapsed_time;

    while (t_cur < t_max) {
        t_cur += tau;

        cudaEventRecord(start);
        /* k(t) = { v(t), a(t) } */
        calc_k<<<(n + BS) / BS, BS>>>(d_k, d_p_cur, d_masses, n);
        /* p(t + tau/2) = p(t) + tau / 2 * k(t) */
        plus_product<<<(size + BS) / BS, BS>>>(d_p_mid, d_p_cur, tau / 2, d_k, size);
        /* k(t + tau/2) = { v(t + tau/2), a(t + tau/2) } */
        calc_k<<<(n + BS) / BS, BS>>>(d_k, d_p_mid, d_masses, n);
        /* p(t + tau) += tau * k(t + tau/2) */
        plus_equal_product<<<(size + BS) / BS, BS>>>(d_p_cur, tau, d_k, size);
        /* end of record */
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&elapsed_time, start, stop);
        exec_time += elapsed_time;

        cudaEventRecord(start);
        /* copy p(t + tau) to host */
        cudaMemcpy(p_cur.data(), d_p_cur, size * sizeof(double), cudaMemcpyDeviceToHost);
        /* end of record */
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&elapsed_time, start, stop);
        copy_time += elapsed_time;

        if (++step_num % print_every_n == 0) {
            stream << setprecision(10) << t_cur << " " << p_cur << '\n';
        }
    }

    cudaDeviceSynchronize();

    cudaFree(d_p_cur);
    cudaFree(d_p_mid);
    cudaFree(d_masses);
    cudaFree(d_k);
//    cout << "Time exec: " << exec_time / 1000.0 << endl;
//    cout << "Time copy: " << copy_time / 1000.0 << endl;
}

int main(int argc, char** argv) {
    // $./run N bs
    if (argc > 1) {
        N = stoi(argv[1]);
    }
    if (argc > 2) {
        BS = stoi(argv[2]);
    }
    cout << "N: " << N << "\nbs: " << BS << "\n";

    /* part 1 data init */
    vector<double> input_data = {1.0, 0.0, 0.0, 0.0, 0.9, 0.0,
                                 0.0, 1.0, 0.0, -0.9, 0.0, 0.0,
                                 -1.0, 0.0, 0.0, 0.0, -0.9, 0.0,
                                 0.0, -1.0, 0.0, 0.9, 0.0, 0.0};
    vector<double> masses = {8810324116.227, 8810324116.227, 8810324116.227, 8810324116.227};
    vector<double> init(input_data.size());
    for (int i = 0; i < init.size() / 6; ++i) {
        init[3 * i + 0] = input_data[6 * i + 0];
        init[3 * i + 1] = input_data[6 * i + 1];
        init[3 * i + 2] = input_data[6 * i + 2];
        init[init.size() / 2 + 3 * i + 0] = input_data[6 * i + 3];
        init[init.size() / 2 + 3 * i + 1] = input_data[6 * i + 4];
        init[init.size() / 2 + 3 * i + 2] = input_data[6 * i + 5];
    }

    /* part 1 calc */
    runge_kutta_2(init, masses, 20, 0.01, 10, "./cuda.txt");

    /* distributions init */
    std::random_device seeder;
    const auto seed = seeder.entropy() ? seeder() : time(nullptr);
    std::mt19937 rnd_gen(static_cast<std::mt19937::result_type>(seed));
    std::uniform_real_distribution<double> distr_pos(-10.0, 10.0);
    std::uniform_real_distribution<double> distr_vel(-1.0, 1.0);
    std::uniform_real_distribution<double> distr_mass(0.1, 10.0);
    auto rnd_pos = std::bind(distr_pos, rnd_gen);
    auto rnd_vel = std::bind(distr_vel, rnd_gen);
    auto rnd_mas = std::bind(distr_mass, rnd_gen);

    /* part 2 data init */
    init.resize(6 * N);
    for (int i = 0; i < 3 * N; ++i) {
        init[i] = rnd_pos();
    }
    for (int i = 3 * N; i < 6 * N; ++i) {
        init[i] = rnd_vel();
    }
    masses.resize(N);
    for (int i = 0; i < N; ++i) {
        masses[i] = rnd_mas();
    }

    /* part 2 calc */
    auto t1 = chrono::high_resolution_clock::now();
    runge_kutta_2(init, masses, 0.1, 0.05, 1, "./cuda_10000.txt");
    auto t2 = chrono::high_resolution_clock::now();
    cout << "Time cuda: " << chrono::duration_cast<chrono::milliseconds>(t2 - t1).count() / 1000.0 << endl;

    return 0;
}

Writing main1.cu


In [13]:
!nvcc -arch=sm_75 -std=c++11 -O2 -o run1 -Wno-deprecated-gpu-targets main1.cu

In [14]:
!./run1

N: 10000
bs: 1024
Time cuda: 0.086


In [20]:
# old
!./run1 10000 32
!echo 
!./run1 10000 1024
!echo 
!./run1 40000 32
!echo 
!./run1 40000 1024

N: 10000
bs: 32
Time cuda: 1.76

N: 10000
bs: 1024
Time cuda: 5.525

N: 40000
bs: 32
Time cuda: 21.292

N: 40000
bs: 1024
Time cuda: 21.45


In [28]:
# pow -> sqr
# pos_vel[3 * idx + k] -> __register__ 
!./run1 10000 32
!echo 
!./run1 10000 1024
!echo 
!./run1 40000 32
!echo 
!./run1 40000 1024

N: 10000
bs: 32
Time cuda: 0.655

N: 10000
bs: 1024
Time cuda: 1.771

N: 40000
bs: 32
Time cuda: 6.488

N: 40000
bs: 1024
Time cuda: 6.485


In [43]:
# pow -> sqr
# pos_vel[3 * idx + k] -> __register__ 
# pos_vel[3 * j + k] -> __shared__
!./run1 10000 32
!echo 
!./run1 10000 1024
!echo
!./run1 20000 32
!echo 
!./run1 20000 1024
!echo 
!./run1 40000 32
!echo 
!./run1 40000 1024

N: 10000
bs: 32
Time cuda: 1.798

N: 10000
bs: 1024
Time cuda: 0.661

N: 20000
bs: 32
Time cuda: 1.884

N: 20000
bs: 1024
Time cuda: 1.108

N: 40000
bs: 32
Time cuda: 1.975

N: 40000
bs: 1024
Time cuda: 1.971


In [3]:
# Tesla K80
!./run1 10000 32
!echo 
!./run1 10000 1024
!echo
!./run1 20000 32
!echo 
!./run1 20000 1024
!echo 
!./run1 40000 32
!echo 
!./run1 40000 1024

N: 10000
bs: 32
Time cuda: 0.263

N: 10000
bs: 1024
Time cuda: 0.198

N: 20000
bs: 32
Time cuda: 0.285

N: 20000
bs: 1024
Time cuda: 0.273

N: 40000
bs: 32
Time cuda: 0.427

N: 40000
bs: 1024
Time cuda: 0.446


In [5]:
240.749 / 0.427, 240.749 / 0.446

(563.8149882903981, 539.7959641255605)